### model time series using LSTM or GRU
* encoder summarizes time series into a latent vector
* decoder reconstructs the time series from it

In [ ]:
class RNNEncoder(nn.Module):
    def __init__(self, input_size, hidden_size, latent_dim):
        super().__init__()
        self.rnn = nn.GRU(input_size, hidden_size, batch_first=True)
        self.fc_mu = nn.Linear(hidden_size, latent_dim)
        self.fc_logvar = nn.Linear(hidden_size, latent_dim)

    def forward(self, x):  # x: (batch, T, N)
        h_seq, h_last = self.rnn(x)
        h = h_last[-1]  # final hidden state
        return self.fc_mu(h), self.fc_logvar(h)


In [ ]:
class RNNDecoder(nn.Module):
    def __init__(self, latent_dim, hidden_size, output_size, seq_len):
        super().__init__()
        self.fc = nn.Linear(latent_dim, hidden_size)
        self.rnn = nn.GRU(output_size, hidden_size, batch_first=True)
        self.out = nn.Linear(hidden_size, output_size)
        self.seq_len = seq_len

    def forward(self, z):
        h0 = self.fc(z).unsqueeze(0)  # (1, batch, hidden)
        input_seq = torch.zeros(z.size(0), self.seq_len, output_size).to(z.device)
        out, _ = self.rnn(input_seq, h0)
        return self.out(out)  # (batch, T, N)
